In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import shap
import sys
import os

# Scikit-learn Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# To import from the `src/` directory, we add it to the Python path.
# This is a robust way to access our pipeline's configuration,
# honoring the project's architecture.
try:
    # Get the project root assuming the notebook is in `notebooks/`
    project_root = os.path.dirname(os.getcwd())
    src_dir = os.path.join(project_root, "src")
    sys.path.append(src_dir)
    from modeling_pipeline import Config

    print("Successfully imported Config from src/modeling_pipeline.py")
except (ImportError, ModuleNotFoundError):
    print(
        "Could not import Config. Please ensure the notebook is in the correct directory."
    )
    # Define a fallback or exit if the import fails
    sys.exit()

c:\Users\johnl\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully imported Config from src/modeling_pipeline.py


In [ ]:
print("--- Preparing the Champion Model: LogisticRegression on Baseline ---")

# Load the baseline dataset using our shared configuration
dataset_path = Config.DATASETS["baseline"]
df = pd.read_csv(dataset_path)

# Prepare data using the central configuration
df[Config.TARGET_VARIABLE] = df[Config.TARGET_VARIABLE].map(
    {"positive": 1, "negative": 0}
)
X = df.drop(columns=[Config.TARGET_VARIABLE])
y = df[Config.TARGET_VARIABLE]

# Create a final, reproducible train/test split for this notebook
# We use a fixed random_state for consistency and stratify due to the data skew
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=Config.TEST_SIZE,
    random_state=42,  # A standard for reproducible splits
    stratify=y,
)

# Dynamically identify feature types, just as our pipeline does
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=np.number).columns.tolist()

# Create the exact same preprocessor to ensure consistency
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ],
    remainder="passthrough",
)

# Initialize and train the final Logistic Regression model within a scikit-learn Pipeline
# This pipeline handles preprocessing and modeling in one clean, architectural step
champion_model_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(random_state=42, max_iter=1000)),
    ]
)

champion_model_pipeline.fit(X_train, y_train)

print("\nChampion model trained successfully within a reproducible pipeline.")